# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet
from pointRegressorSurrogateModel import PointRegressor

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load test data

In [2]:
dataDir = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/"
testFile = os.path.join(dataDir, 'design_7_N_1000.csv')
allGraphsUnfiltered = loadGhGraphs(testFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
testGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max()
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 2. Load pre-train data

In [4]:
pretrainFiles = glob.glob(os.path.join(dataDir, '*1000.csv'))
pretrainFiles.remove(testFile)

allPretrainGraphs = []
for pretrainFile in pretrainFiles:
    designName = pretrainFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    graphsUnfiltered = loadGhGraphs(pretrainFile, NUM_DV=5)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, test = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading design_9
loading design_6
loading design_8
loading design_5
loaded 3600 pretraining graphs


## 3. Initial training

In [ ]:
saveDir = './results/transferLrn01/'
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(allPretrainGraphs, valData, 
                         epochs=100, 
                         batch_size=256, 
                         flatten=True, 
                         logTrans=False, 
                         ssTrans=True, 
                         saveDir=saveDir+f'preTrain/gcn/)

plotHistory(history)

## 4. Transfer learning study

In [ ]:
dataDir = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/"
trainDataFiles = glob.glob(os.path.join(dataDir, 'design_7*.csv'))
trainDataFiles.remove(doeFile)

allResults = []
for trainDataFile in trainDataFiles:
    trainData = loadGhGraphs(trainDataFile, NUM_DV=5)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                     flatten=False, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir=saveDir+f'{trainSize}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['set'] = 'Train'
    trainRes['train size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['set'] = 'Test'
    testRes['train size'] = trainSize
    allResults.append(testRes)
    
pd.DataFrame(allResults)